In [13]:
import os
import sys
current_directory = os.path.dirname(sys.argv[0])

PHOTO_PATH = current_directory + "/data"
INPUT_FILE = current_directory + "/image.jpg"
OUTPUT_FILE = current_directory + "/result.txt"
MODEL_PATH = current_directory + "/model"
CURRENT_DIRECTORY = current_directory

MINIMUM_PHOTOS = 5

In [14]:
import cv2
import numpy as np
from PIL.Image import fromarray

class ImagePreprocessor:
    def __init__(self):
        self._faceCascade = cv2.CascadeClassifier(constants.CURRENT_DIRECTORY + "/haarcascade_frontalface_default.xml")

    def convert_image(self, image):
        image = image.convert('L')
        image = np.array(image, 'uint8')
        faces = self._faceCascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            return fromarray(image[y: y + h, x: x + w]).resize((50, 50))

        return None

    def filter_images(self, X, Y):
        X_result = []
        Y_result = []

        for x, y in zip(X, Y):
            x_row = []

            for row in x:
                x_row = x_row + row.tolist()

            X_result.append(x_row)
            Y_result.append(y)

        return X_result, Y_result

ModuleNotFoundError: No module named 'cv2'

In [15]:
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from sklearn.svm import SVC

class FaceRecognizer:
    def __init__(self):
        self._pca = PCA(n_components=300, whiten=True)
        self._pipeline = (
            SVC(kernel='rbf', class_weight='balanced', C=5, gamma=0.001, probability=True, tol=0.0000001),
            SVC(kernel='rbf', class_weight='balanced', C=10, gamma=0.0001, probability=True, tol=0.00000001),
        )

        self._classes = dict()
        self._classes_reverse = dict()

    def fit(self, X_train, y_train):
        self._pca.fit(X_train)

        for y in y_train:
            if not self._classes.get(y):
                self._classes[y] = len(self._classes) + 1
                self._classes_reverse[len(self._classes)] = y

        y_train_transform = []
        for y in y_train:
            y_train_transform.append(self._classes[y])

        X_train = self._pca.transform(X_train)

        for pipe in self._pipeline:
            pipe.fit(X_train, y_train_transform)

    def reverse_y(self, Y):
        return [self._classes[y] for y in Y]

    def get_target_names(self):
        return [self._classes_reverse[i] for i in range(1, len(self._classes) + 1)]

    def predict(self, X_test):
        pipe_predict = self.predict_proba(X_test)

        y_predict = []
        for y_pred in pipe_predict:
            y_predict.append(y_pred.index(max(y_pred)) + 1)

        return y_predict

    def predict_proba(self, X_test):
        X_test = self._pca.transform(X_test)
        pipe_predict = [[0] * len(self._classes) for _ in range(len(X_test))]

        for pipe in self._pipeline:
            prediction = pipe.predict_proba(X_test).tolist()

            for i in range(len(pipe_predict)):
                for j in range(len(pipe_predict[i])):
                    pipe_predict[i][j] = max(pipe_predict[i][j], prediction[i][j])

        return pipe_predict

    def save(self, path):
        joblib.dump(self, path)

    @staticmethod
    def load(path):
        return joblib.load(path)


In [16]:
from PIL import Image

def main():
    image_preprocessor = ImagePreprocessor()

    X_load = []
    Y_load = []

    classes = dict()

    for path in os.listdir(constants.PHOTO_PATH):
        classes[path] = 0
        for file_name in os.listdir(constants.PHOTO_PATH + "//" + path):
            file_path = PHOTO_PATH + "//" + path + "//" + file_name
            image = image_preprocessor.convert_image(Image.open(file_path))
            if not image:
                continue

            image = np.array(image, 'uint8')
            classes[path] += 1
            X_load.append(image)
            Y_load.append(path)

    X = []
    Y = []

    for x, y in zip(X_load, Y_load):
        if classes[y] < constants.MINIMUM_PHOTOS:
            continue

        X.append(x)
        Y.append(y)

    X, Y = image_preprocessor.filter_images(X, Y)

    face_recognizer = FaceRecognizer()
    face_recognizer.fit(X, Y)
    face_recognizer.save(MODEL_PATH)


if __name__ == "__main__":
    main()


NameError: name 'cv2' is not defined

In [20]:
from PIL import Image

def main():
    face_recognizer = FaceRecognizer().load(MODEL_PATH)
    image = Image.open(INPUT_FILE)
    image = ImagePreprocessor().convert_image(image)

    if not image:
        result_file = open(OUTPUT_FILE, "w")
        result_file.write("NOT FOUND")
        result_file.close()
        return

    image = np.array(image, 'uint8')

    image, y = ImagePreprocessor().filter_images([image], [1])

    probability = face_recognizer.predict_proba(image)[0]

    result_file = open(OUTPUT_FILE, "w")

    if max(probability) >= 0.6:
        answer = probability.index(max(probability))
        result_file.write(face_recognizer.get_target_names()[answer] + "\n")
        result_file.write(str(max(probability)))

    else:
        result_file.write("NOT FOUND")

    result_file.close()


if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\-\\Anaconda3\\lib\\site-packages/model'